In [3]:
from kan2 import *
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import scipy.stats.stats as st
import csv
df = pd.read_csv('mic_b.csv')
from sklearn.model_selection import train_test_split

protein_list = df['sequence']
y_label = df['mic_value']


def calculate_aac(protein_list):
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

    # 初始化存储结果的列表
    aac_list = []

    for protein_sequence in protein_list:
        # 初始化AAC特征字典
        aac = {aa: 0 for aa in amino_acids}

        # 计算每种氨基酸在序列中的频率
        for aa in protein_sequence:
            if aa in aac:
                aac[aa] += 1

        # 将频率转换为比例
        sequence_length = len(protein_sequence)
        aac = [count / sequence_length for aa, count in aac.items()]

        # 将结果添加到列表中
        aac_list.append(aac)

    return aac_list


aac = calculate_aac(protein_list)


def aaindex(protein_list):
    # protein_list = []
    csv_file_path = 'aaindex1.csv'
    lag = 2
    data = []
    with open(csv_file_path, mode='r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)

        # Skip the first cell of the first row
        for row in csv_reader:
            # Append the row with the first cell removed to the data list
            # data.append([float(value) for value in row[1:]])
            if 'A' in row or 'NA' in row:
                continue
            row = row[1:]
            data.append([float(value) for value in row])
    data = data[1:]
    # print(data[0])
    AAindex_list = pd.DataFrame(data)

    # 用0填充 NaN 值（或者你可以选择其他处理方法）
    AAindex_list = AAindex_list.fillna(0)

    # 将 DataFrame 转换回二维列表
    AAindex_list = AAindex_list.values.tolist()
    if np.isnan(AAindex_list).any() or np.isinf(AAindex_list).any():
        print("1数组中含有 NaN 或无穷大的值")

    autocorrelation = []
    for sequence in protein_list:
        # print(sequence)
        temp = []
        for property_values in AAindex_list:
            # 将氨基酸序列转换为属性值序列
            #     property_values = np.array([property_dict[aa] for aa in sequence])
            property_values = [0 if value is None else value for value in property_values]
            # 计算属性值的平均值
            # print(property_values)
            mean_value = np.mean(property_values)
            # print(mean_value)
            # 计算Moran自相关
            n = len(sequence)
            if n <= lag:
                temp.append(0)
                continue
            autocorr = np.sum((property_values[:-lag] - mean_value) * (property_values[lag:] - mean_value))
            autocorr /= (n - lag)
            # print(autocorr)
            temp.append(autocorr)

        autocorrelation.append(temp)
    if np.isnan(autocorrelation).any() or np.isinf(autocorrelation).any():
        print("2数组中含有 NaN 或无穷大的值")

    v = []
    for i in range(len(autocorrelation)):
        vtar = autocorrelation[i]
        vtarv = []
        vtar7 = 0
        vtar8 = 0
        vtar9 = 0
        s = pd.Series(vtar)
        vtar3 = np.mean(vtar)  # These 4 dimensions are relevant statistical terms
        vtar4 = st.kurtosis(vtar)
        vtar5 = np.var(vtar)
        vtar6 = st.skew(vtar)
        #for p in range(len(vtar)): # These 3 dimensions are inspired by PAFIG algorithm
        #vtar7=vtar[p]**2+vtar7
        #if vtar[p]>va:
        #vtar8=vtar[p]**2+vtar8
        #else:
        #vtar9=vtar[p]**2+vtar9
        vcf1 = []
        vcf2 = []
        for j in range(len(vtar) - 1):  #Sequence-order-correlation terms
            vcf1.append((vtar[j] - vtar[j + 1]))
        for k in range(len(vtar) - 2):
            vcf2.append((vtar[k] - vtar[k + 2]))
        vtar10 = np.mean(vcf1)
        vtar11 = np.var(vcf1)
        vtar11A = st.kurtosis(vcf1)
        vtar11B = st.skew(vcf1)
        vtar12 = np.mean(vcf2)
        vtar13 = np.var(vcf2)
        vtar13A = st.kurtosis(vcf2)
        vtar13B = st.skew(vcf2)
        vtarv.append(vtar3)
        vtarv.append(vtar4)
        vtarv.append(vtar5)
        vtarv.append(vtar6)
        #vtarv.append(vtar7/len(vtar))
        #vtarv.append(vtar8/len(vtar))
        #vtarv.append(vtar9/len(vtar))
        vtarv.append(vtar10)
        vtarv.append(vtar11)
        vtarv.append(vtar11A)
        vtarv.append(vtar11B)
        vtarv.append(vtar12)
        vtarv.append(vtar13)
        vtarv.append(vtar13A)
        vtarv.append(vtar13B)
        v.append(vtarv)
    return v


AC = aaindex(protein_list)
AC = np.array(AC)  # 转换为 NumPy 数组

# 检查是否为二维数组
if len(AC.shape) == 2:
    # 对每一行进行 Z-score 标准化
    AC = (AC - np.mean(AC, axis=1, keepdims=True)) / np.std(AC, axis=1, keepdims=True)
    print(len(AC))
else:
    print("AC 不是二维数组")
# print(AC,AC.shape)

if np.isnan(AC).any() or np.isinf(AC).any():
    print("数组中含有 NaN 或无穷大的值")

    # 找出NaN的位置
    nan_indices = np.where(np.isnan(AC))[0]  # 找出 NaN 的索引
    print("NaN值所在的索引:", nan_indices)

    # 找出无穷大的位置
    inf_indices = np.where(np.isinf(AC))[0]  # 找出 inf 的索引
    print("无穷大值所在的索引:", inf_indices)
else:
    print("数组中没有 NaN 或无穷大的值")


def maccs_feature(file):
    maccs_list = []
    with open(file, 'r') as f:
        for line in f:
            maccs = list(map(int, line.strip().split()))
            # print(maccs)
            maccs_list.append(maccs)
    return maccs_list


train_file = 'maccs_AMP.txt'
maccs = maccs_feature(train_file)

# len(maccs),maccs[0]
X = np.concatenate((maccs, aac, AC), axis=1)
# X=np.array(aac)
X = np.array(X)
y = np.array(y_label)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # 将 DataFrame 和 Series 转换为 np.array# 数据集标准化
# x_mean = X_train.mean()
# x_std =  X_train.std()
# y_mean = y.mean()
# y_std = y.std()
# X_train = (X_train - x_mean)/x_std
# y_train = (y_train-y_mean)/y_std
# X_test = (X_test - x_mean)/x_std
# y_test = (y_test - y_mean)/y_std
import torch

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
dataset = {}
dataset = {
    'train_input': X_train,
    'test_input': X_test,
    'train_label': y_train,
    'test_label': y_test
}
X_train.shape

6083
数组中没有 NaN 或无穷大的值


/home/wangli/anaconda3/envs/mykan/lib/python3.7/site-packages/ipykernel_launcher.py:213: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/wangli/anaconda3/envs/mykan/lib/python3.7/site-packages/ipykernel_launcher.py:214: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/wangli/anaconda3/envs/mykan/lib/python3.7/site-packages/ipykernel_launcher.py:215: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/wangli/anaconda3/envs/mykan/lib/python3.7/site-packages/ipykernel_launcher.py:216: UserWarning: To copy construct from a tensor, it is reco

torch.Size([4866, 199])

In [4]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive'],
}

# 创建 MLPRegressor 模型
mlp = MLPRegressor(max_iter=100, random_state=42)

# 创建 GridSearchCV 对象
grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)

# 训练模型
grid_search.fit(X_train, y_train)

# 查看最佳参数组合
print("Best parameters found: ", grid_search.best_params_)

import pickle

# 获取 GridSearchCV 中的最佳模型
best_model = grid_search.best_estimator_

# 保存模型到文件
model_path = 'best_mlp_model.pkl'
with open(model_path, 'wb') as file:
    pickle.dump(best_model, file)

print(f"Model saved to {model_path}")

# 使用最佳模型进行预测
y_pred = grid_search.predict(X_test)
print(y_pred,y_train)
# 计算评估指标
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100  # MAPE计算

# 输出评估指标
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R-squared (R2): {r2:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.4f}%")

Fitting 5 folds for each of 96 candidates, totalling 480 fits


/home/wangli/anaconda3/envs/mykan/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/home/wangli/anaconda3/envs/mykan/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/home/wangli/anaconda3/envs/mykan/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/home/wangli/anaconda3/envs/mykan/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet

Best parameters found:  {'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (100, 50), 'learning_rate': 'constant', 'solver': 'adam'}
Model saved to best_mlp_model.pkl
[1.52385257 0.92454369 1.284129   ... 3.08676691 1.31151541 0.54374201] [-0.98944236  2.          0.78701563 ...  1.15514723  2.27004404
  2.        ]
Mean Squared Error (MSE): 0.3924
Root Mean Squared Error (RMSE): 0.6264
Mean Absolute Error (MAE): 0.4999
R-squared (R2): 0.3144
Mean Absolute Percentage Error (MAPE): inf%


/home/wangli/anaconda3/envs/mykan/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/home/wangli/anaconda3/envs/mykan/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: divide by zero encountered in true_divide


In [ ]:
import torch
from torch import nn, optim
from kan2 import *
# 定义回归MLP模型
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )
        
    def forward(self, x):
        return self.layers(x)

# 创建并加载MLP模型（适用于回归）
mlp = MLP(input_size=4, hidden_size=100, output_size=1)
mlp.load_state_dict(torch.load('path_to_regression_mlp_model.pth'))  # 加载训练好的MLP模型
mlp.eval()  # 设置为评估模式

# 创建KAN模型
model = KAN(width=[4, 5, 3], grid=5, k=3, seed=0, device='cpu')


In [ ]:
def distillation_loss(output, target_output, target, alpha=0.5):
    """ 回归任务的蒸馏损失函数 """
    teacher_loss = nn.MSELoss()(output, target_output)
    student_loss = nn.MSELoss()(output, target)
    return alpha * teacher_loss + (1 - alpha) * student_loss
